In [1]:
"""
!pip install tensorboard
!pip install accelerate -U


!pip install scipy
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install ipywidgets
"""

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-op5ccgxs
  Running command git clone --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-op5ccgxs
  Resolved https://github.com/huggingface/transformers to commit 5bb4430edc7df9f9950d412d98bbe505cc4d328b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, r


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [17]:

import torch
if torch.cuda.is_available(): 
   device_count = torch.cuda.device_count()
   for i in range(device_count):
       print('Device:', i, ' ', torch.cuda.get_device_name(i))
else:
   print('No GPU available')



Device: 0   NVIDIA A40


In [12]:
%%writefile whisper4fsegmentation_focus_prototype__offline.py
import sys

import torch

if torch.cuda.is_available(): 
   device_count = torch.cuda.device_count()
   for i in range(device_count):
       print('Device:', i, ' ', torch.cuda.get_device_name(i))
else:
   print('No GPU available')







import pickle
# Load the list from the file
with open('/home/projects/dharel/eranbe/Eran/all_data_list_poc_segprotfoc_include21.pkl', 'rb') as f:
    all_data_list = pickle.load(f)





import pandas as pd
import numpy as np


df = pd.DataFrame(all_data_list)

for index, row in df.iterrows():
    row['audio']['array'] = np.array(row['audio']['array'], dtype=np.float32)


from datasets import Dataset, DatasetDict


#NEED TO SHUFFLE!!

# Fix the random seed for shuffling
seed = 42  # Or any other number you like

train_df = df.iloc[:int(0.95*len(df))]
train_df = train_df.sample(frac=1, random_state=seed).reset_index(drop=True)  # shuffle and reset index
train_dataset = Dataset.from_pandas(train_df)



# Create the Dataset objects
#train_dataset = Dataset.from_pandas(df.iloc[:int(0.95*len(df))])




test_dataset = Dataset.from_pandas(df.iloc[int(0.95*len(df)):])

# Create the DatasetDict object
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")

from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="english", task="transcribe")


from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2", language="english", task="transcribe")


#need to trancate sequence ,model works only with 1024 token.
#@@@@@@@@@@
#@@@@@@@@@@
#$$$$$$$$$$$$$$$$$
#MAX_LENGTH = 1024
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # truncate target text to a maximum length
    #sentence = batch["sentence"][:MAX_LENGTH]
    sentence = batch["sentence"]
        
    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    # encode target text to label ids 
    labels_ids = tokenizer(sentence).input_ids
    # truncate labels to a maximum length
    #labels_ids = labels_ids[:MAX_LENGTH]
    batch["labels"] = labels_ids
    return batch


dataset = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["train"], num_proc=4)
max_input_length = 30
min_input_length = 0


def is_audio_in_length_range(length):
    return length > min_input_length and length <= max_input_length

vectorized_datasets = dataset.filter(
    is_audio_in_length_range, num_proc=4, input_columns=["input_length"]
)

vectorized_datasets
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")

model.config.max_length
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")

model.config.suppress_tokens = []
max_label_length = model.config.max_length
max_label_length
def is_labels_in_length_range(labels):
    return len(labels) < max_label_length
vectorized_datasets = vectorized_datasets.filter(
    is_labels_in_length_range, num_proc=4, input_columns=["labels"]
)

vectorized_datasets
common_voice = vectorized_datasets


import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
    
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir = "/home/projects/dharel/eranbe/Eran/whisper_seg_focus_prototype_wexac_17_07_2023_v2booya_offline",
    #output_dir="/content/drive/MyDrive/Weizmann/SBC1to60/whisper-large-v2-English2segmentation_v1_2023",  # change to a repo name of your choice
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=6000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


processor.save_pretrained(training_args.output_dir)

trainer.train()


print(f"your argument is {sys.argv[1]}")

Overwriting whisper4fsegmentation_focus_prototype__offline.py


In [55]:
%%writefile whisper4fsegmentation_focus_prototype__offline.sh
bsub <<EOT
#!/bin/bash
#BSUB -gpu num=1:j_exclusive=yes:gmem=30G
#BSUB -R rusage[mem=32000]
#BSUB -q gpu-long -m hgn_2022_hosts gmodel=NVIDIAA100
#BSUB --job-name=booya # Job name
#BSUB --output=/home/projects/dharel/eranbe/Eran/whisper4fsegmentation_focus_prototype__offline${1}.log # Standard output and error log (mind the log)
#BSUB --time=24:00:00 # Time limitation (here 15 minutes)
#BSUB --mail-user=your@mail # Email address for notification sending
#BSUB --mail-type=END,FAIL # When to send email notification

python /home/projects/dharel/eranbe/Eran/whisper4fsegmentation_focus_prototype__offline.py ${1}
EOT

Overwriting whisper4fsegmentation_focus_prototype__offline.sh


In [56]:
!bash whisper4fsegmentation_focus_prototype__offline.sh "6000_v3"

Please note that memory reservation is per task (-n X)
Memory reservation is (MB): 32000 
===Your total amount of memory reservation for this job is (MB): 32000 ===
Job <673451> is submitted to queue <gpu-long>.


In [2]:
!bjobs

JOBID      USER    STAT  QUEUE      FROM_HOST   EXEC_HOST   JOB_NAME   SUBMIT_TIME
920068     eranbe  RUN   new-intera access1     cn653       *upyterhub Jul 21 12:44
927269     eranbe  RUN   gpu-intera access      gaccess07   *upyterhub Jul 21 18:11


In [4]:
!bpeek 920058

Job <920058> is not found


In [80]:
!bjobs -l -gpu 673451


Job <673451>, User <eranbe>, Project <default>, User Group <dharel-wx-grp-lsf>,
                     Status <RUN>, Queue <gpu-long>, Job Priority <50>, Command
                      <#!/bin/bash;#BSUB -gpu num=1:j_exclusive=yes:gmem=38G;#B
                     SUB -R rusage[mem=32000];#BSUB -q gpu-long -m hgn_2022_hos
                     ts gmodel=NVIDIAA100;#BSUB --job-name=booya # Job name;#BS
                     UB --output=/home/projects/dharel/eranbe/Eran/whisper4fseg
                     mentation_focus_prototype__offline6000_v3.log # Standard o
                     utput and error log (mind the log);#BSUB --time=24:00:00 #
                      Time limitation (here 15 minutes);#BSUB --mail-user=your@
                     mail # Email address for notification sending;#BSUB --mail
                     -type=END,FAIL # When to send email notification; python /
                     home/projects/dharel/eranbe/Eran/whisper4fsegmentation_foc
                     us_prototype__offl

In [35]:
!bjobs -l -gpu 673396


Job <673396>, User <eranbe>, Project <default>, User Group <dharel-wx-grp-lsf>,
                     Status <PEND>, Queue <gpu-long>, Job Priority <52>, Comman
                     d <#!/bin/bash;#BSUB -gpu num=1:j_exclusive=yes:gmem=46G;#
                     BSUB -R rusage[mem=32000];#BSUB -q gpu-long -m hgn_2022_ho
                     sts gmodel=NVIDIAA100;#BSUB --job-name=booya # Job name;#B
                     SUB --output=/home/projects/dharel/eranbe/Eran/whisper4fse
                     gmentation_focus_prototype__offline6000_v3.log # Standard 
                     output and error log (mind the log);#BSUB --time=12:00:00 
                     # Time limitation (here 15 minutes);#BSUB --mail-user=your
                     @mail # Email address for notification sending;#BSUB --mai
                     l-type=END,FAIL # When to send email notification; python 
                     /home/projects/dharel/eranbe/Eran/whisper4fsegmentation_fo
                     cus_prototype__off

In [83]:
!bkill asda

Job <673451> is being terminated


In [225]:
!blimits -q gsla-gpu

No resource usage found.


In [231]:
# INTERACTIVE

In [19]:
%%writefile whisper4fsegmentation_focus_prototype__offline.sh
bsub <<EOT
#!/bin/bash
#BSUB -gpu num=1:j_exclusive=yes:gmem=46G
#BSUB -R rusage[mem=32000]
#BSUB -q gpu-interactive -m hgn_2022_hosts gmodel=NVIDIAA40
#BSUB --job-name=booya # Job name
#BSUB --output=/home/projects/dharel/eranbe/Eran/whisper4fsegmentation_focus_prototype__offline${1}.log # Standard output and error log (mind the log)
#BSUB --time=12:00:00 # Time limitation (here 15 minutes)
#BSUB --mail-user=your@mail # Email address for notification sending
#BSUB --mail-type=END,FAIL # When to send email notification

python /home/projects/dharel/eranbe/Eran/whisper4fsegmentation_focus_prototype__offline.py ${1}
EOT

Overwriting whisper4fsegmentation_focus_prototype__offline.sh


In [20]:
!bash whisper4fsegmentation_focus_prototype__offline.sh "6000_v3"

Memory reservation is (MB): 32000
Memory Limit is (MB): 32000

===Your total amount of memory reservation for this job is (MB): 32000 ===

Job <673408> is submitted to queue <gpu-interactive>.


In [36]:
!bjobs

JOBID      USER    STAT  QUEUE      FROM_HOST   EXEC_HOST   JOB_NAME   SUBMIT_TIME
673243     eranbe  RUN   new-intera access1     cn651       *upyterhub Jul 17 14:30
673408     eranbe  PEND  gpu-intera cn651                   *y 6000_v3 Jul 17 14:44
673396     eranbe  PEND  gpu-long   cn651                   *y 6000_v3 Jul 17 14:42


In [28]:
!bpeek 673408

Job <673408> : Not yet started.


In [29]:
!bjobs -p2 673408

JOBID      USER    STAT  QUEUE      FROM_HOST   EXEC_HOST   JOB_NAME   SUBMIT_TIME
673408     eranbe  PEND  gpu-intera cn651                   *y 6000_v3 Jul 17 14:44
 Candidate host pending reasons (15 of 546 hosts):
   Not enough GPUs with the required memory on the hosts: 11 hosts;
   Job's requirements for resource reservation not satisfied (Resource: mem): 2 hosts;
   Host's available GPU resources cannot meet the job's requirements: 1 host;
   Job's requirements for resource reservation not satisfied (Resource: ngpus_physical): 1 host;


In [30]:
!bjobs -l -gpu 673408


Job <673408>, User <eranbe>, Project <default>, User Group <dharel-wx-grp-lsf>,
                     Status <PEND>, Queue <gpu-interactive>, Job Priority <50>,
                     Command <#!/bin/bash;#BSUB -gpu num=1:j_exclusive=yes:gmem
                     =46G;#BSUB -R rusage[mem=32000];#BSUB -q gpu-interactive -
                     m hgn_2022_hosts gmodel=NVIDIAA40;#BSUB --job-name=booya #
                      Job name;#BSUB --output=/home/projects/dharel/eranbe/Eran
                     /whisper4fsegmentation_focus_prototype__offline6000_v3.log
                      # Standard output and error log (mind the log);#BSUB --ti
                     me=12:00:00 # Time limitation (here 15 minutes);#BSUB --ma
                     il-user=your@mail # Email address for notification sending
                     ;#BSUB --mail-type=END,FAIL # When to send email notificat
                     ion; python /home/projects/dharel/eranbe/Eran/whisper4fseg
                     mentation_focus_pr

In [ ]:
!bkill 668984

In [1]:
#from datasets import Audio

#dataset_dict = dataset_dict.cast_column("audio", Audio(sampling_rate=16000))